In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle, os

# 모델 파일 경로
model_filepath = '../data/훈련용/best_model_v1.pkl'

# 모델 불러오기
with open(model_filepath, 'rb') as file:
    classification_model = pickle.load(file)

classification_model

SVC(C=14547.641687966585, cache_size=43605.932291666664,
    class_weight='balanced', gamma=0.027760342173282767, max_iter=-1.0,
    random_state=1, shrinking=False, tol=2.6344220036014925e-05)

In [8]:
# 미래 온습도 데이터, 대기질 데이터 호출
future_onssp_DIR = "../data/기온_습도_미세먼지 데이터/미래예측결과"
future_air_DIR = "../data/미래대기질데이터"

FUTURE_AIR_FILES = os.listdir(future_air_DIR)
FUTURE_ONSSP_FILES = os.listdir(future_onssp_DIR)

len(FUTURE_AIR_FILES) == len(FUTURE_ONSSP_FILES)

True

[ SO2 | CO | O3 | NO2 | PM10 | PM25 | 지역명 | year | month | day | 도로명 | 기온 | 습도 | 평균속도 | 최고속도 | weekday ]

In [28]:
base_df = pd.read_csv('../data/전처리 완료/dataset.csv')[['지역명', '도로명']].drop_duplicates()
base_df 

,지역명,도로명
0,강서구,가로공원로
2,양천구,가로공원로
32,구로구,가마산로
33,영등포구,가마산로
65,금천구,가산디지털1로
...,...,...
6619,동대문구,황물로
6641,동대문구,회기로
6663,서초구,효령로
6687,종로구,효자로


In [47]:
from datetime import datetime, timedelta

# 여기에 예측된 값들도 전송하면 됨. 이후에 예측
def getNewDF(loca, rod):
    # 오늘 날짜
    start_date = datetime.today()

    # 2개월 후 날짜 계산
    end_date = start_date + timedelta(days=60)

    # 날짜 포맷
    date_format = "%Y-%m-%d"

    # 현재 날짜 초기화
    current_date = start_date

    year_col = []
    month_col = []
    day_col = []
    weekday_col = []

    # 반복문을 통해 날짜와 요일 출력
    while current_date <= end_date:
        # YYYY-MM-DD 형식으로 날짜 출력
        date_str = current_date.strftime(date_format) 
        YYYY = date_str[:4]
        MM = date_str[5:7]
        DD = date_str[8:10]
        # 요일 숫자로 출력 (0=월요일, 6=일요일)
        weekday_num = current_date.weekday()

        year_col.append(YYYY)
        month_col.append(MM)
        day_col.append(DD)
        weekday_col.append(weekday_num)

        # print(f"{YYYY}-{MM}-{DD} ({weekday_num})")
        # 다음 날로 이동
        current_date += timedelta(days=1)

    newdf = pd.DataFrame({
        'SO2': np.nan,
        'CO': np.nan,
        'O3': np.nan,
        'NO2': np.nan,
        'PM10': np.nan,
        'PM25': np.nan,
        '지역명': loca,
        'year': year_col,
        'month': month_col,
        'day': day_col,
        '도로명': rod,
        '기온': np.nan,
        '습도': np.nan,
        '평균속도': np.nan,
        '최고속도': np.nan,
        'weekday': weekday_col
    })
    return newdf

df를 저 위대로 만든다고 하고 도로별, 지역별로 각각 만들어서 mapping?

In [37]:
predict_dict = dict()

base_df = pd.read_csv('../data/훈련용/X.csv')
locations = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
        '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
        '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']
roads = ['가로공원로', '가마산로', '가산디지털1로', '가산디지털2로', '가좌로', '강남대로', '강동대로',
        '강서로', '개봉로', '개포로', '개화동로', '거마로', '겸재로', '경인로', '고덕로', '고무래로',
        '고산자로', '고척로', '공릉로', '공원로', '공항대로', '과천대로', '관악로', '광나루로', '광평로',
        '구로동로', '구로중앙로', '구의강변로', '국제금융로', '국회대로', '금낭화로', '금오로', '금하로',
        '긴고랑로', '낙성대로', '난계로', '난곡로', '남대문로', '남부순환로', '노량진로', '노원로',
        '노해로', '녹사평대로', '논현로', '능동로', '다산로', '답십리로', '당산로', '대림로', '대방천로',
        '대학로', '대흥로', '덕릉로', '도곡로', '도당로', '도림로', '도봉로', '도산대로', '도신로',
        '도영로', '독막로', '독산로', '독서당로', '돌곶이로', '동교로', '동남로', '동소문로', '동일로',
        '동작대로', '동호로', '등촌로', '디지털로', '뚝섬로', '마곡중앙5로', '마곡중앙로', '마들로',
        '마장로', '마천로', '마포대로', '만리재로', '망우로', '매헌로', '면목천로', '명달로', '모래내로',
        '목동남로', '목동동로', '목동로', '목동로3길', '목동서로', '목동중앙로', '무학로', '문래로',
        '문성로', '문정로', '바우뫼로', '반포대로', '밤고개로', '밤고개로21길', '방배로', '방학로',
        '방화대로', '백련산로', '백범로', '백제고분로', '버드나루로', '범안로', '벚꽃로', '보국문로',
        '보문로', '봉우재로', '봉은사로', '봉천로', '봉화산로', '부일로', '북한산로', '사가정로', '사당로',
        '사임당로', '사직로', '사평대로', '살곶이길', '삼성로', '삼양로', '삼일대로', '삼전로', '삼학사로',
        '상계로', '상도로', '상봉로', '상일로', '새말로', '새문안로', '새창로', '서강로', '서빙고로',
        '서소문로', '서오릉로', '서운로', '서울시립대로', '서초대로', '서초중앙로', '서해안로', '석촌호수로',
        '선릉로', '선유로', '섬밭로', '성내로', '성북로', '성산로', '성암로', '성현로', '세검정로',
        '세종대로', '소월로', '솔샘로', '송이로', '송파대로', '수락산로', '수명로', '수색로', '숙선옹주로',
        '시흥대로', '신길로', '신내로', '신내역로', '신도림로', '신림로', '신목로', '신반포로', '신사로',
        '신월로', '신정로', '신정로7길', '신정이펜1로', '신촌로', '신풍로', '쑥고개로', '아리랑로',
        '아차산로', '안암로', '안양천로', '압구정로', '양녕로', '양재대로', '양천로', '양평로', '양화로',
        '언주로', '여의공원로', '여의나루로', '여의대방로', '여의동로', '여의서로', '역삼로', '연남로',
        '연서로', '연희로', '영동대로', '영등포로', '영신로', '영중로', '오금로', '오리로', '오목로',
        '오패산로', '올림픽로', '올림픽로35길', '와우산로', '왕산로', '왕십리로', '용마산로', '원효로',
        '월계로', '월드컵로', '월드컵북로', '위례성대로', '율곡로', '은천로', '은평로', '은행나무로',
        '을지로', '응암로', '의사당대로', '이문로', '이촌로', '이태원로', '인수봉로', '인촌로', '일원로',
        '자곡로', '자양로', '자하문로', '잔다리로', '잠실로', '잠원로', '장승배기로', '장안벚꽃로',
        '장충단로', '장한로', '전농로', '정릉로', '제기로', '종로', '종암로', '중계로', '중대로',
        '중랑천로', '중앙로', '증산로', '지봉로', '진관1로', '진흥로', '창경궁로', '창의문로', '창전로',
        '천왕로', '천중로', '천호대로', '청계천로', '청파로', '초안산로', '초원로', '충민로', '태봉로',
        '테헤란로', '토정로', '통일로', '퇴계로', '평창문화로', '풍산로', '하늘길', '학동로', '한강대로',
        '한글비석로', '한남대로', '한천로', '한천로58길', '해등로', '행당로', '허준로', '헌릉로',
        '현충로', '호암로', '홍릉로', '화곡로', '화랑로', '황물로', '회기로', '효령로', '효자로',
        '흑석한강로']


# 이 딕셔너리들 나중에 인코딩에 써먹음.
location_dict = dict()
location_encoding = dict()
road_dict = dict()
road_encoding = dict()

for loc in range(len(locations)):
    location_dict[locations[loc]] = dict()
    location_encoding[locations[loc]] = loc
for road in range(len(roads)):
    road_dict[roads[road]] = dict()
    road_encoding[roads[road]] = road
 

In [38]:
location_dict['강남구']

{}

In [48]:
base_df = pd.read_csv('../data/전처리 완료/dataset.csv')[['지역명', '도로명']].drop_duplicates()
base_df 

,지역명,도로명
0,강서구,가로공원로
2,양천구,가로공원로
32,구로구,가마산로
33,영등포구,가마산로
65,금천구,가산디지털1로
...,...,...
6619,동대문구,황물로
6641,동대문구,회기로
6663,서초구,효령로
6687,종로구,효자로


In [49]:
base_df[base_df['지역명'] == '강남구']

,지역명,도로명
210,강남구,개포로
544,강남구,광평로
840,강남구,남부순환로
1094,강남구,논현로
1362,강남구,도곡로
1450,강남구,도산대로
2409,강남구,밤고개로
2431,강남구,밤고개로21길
2715,강남구,봉은사로
2930,강남구,삼성로


In [54]:
for loc in locations:
    roads_series = base_df[base_df['지역명'] == loc]['도로명']
    for rs in roads_series:
        # print(rs)
        location_dict[loc][rs] = getNewDF(loc, rs)

location_dict['서초구'].keys()

dict_keys(['강남대로', '고무래로', '남부순환로', '논현로', '동작대로', '매헌로', '명달로', '바우뫼로', '반포대로', '방배로', '사임당로', '사평대로', '서운로', '서초대로', '서초중앙로', '신반포로', '양재대로', '잠원로', '태봉로', '헌릉로', '효령로'])

# NEWDF 모두 완료되면 분류기 돌려서 DF에 합쳐라